In [ ]:
# @title Setup
from google.colab import auth
# from google.cloud import bigquery
from google.colab import data_table
import networkx as nx
import pandas as pd
import pickle
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime, timedelta

# Links Data


In [ ]:
links_df = pd.read_parquet(f'/content/drive/My Drive/farcaster_bigquery/links/links.parquet')
# links_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/links.csv')

In [ ]:
links_df = links_df[links_df.type.isin(['follow', 'unfollow'])]
links_df = links_df.drop_duplicates(['target_fid', 'fid'], keep=False)
# links_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/csv/links_followsOnly.csv')
# links_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/parquet/links_followsOnly.parquet')

In [ ]:
links_df.head()

,fid,timestamp,target_fid,type,deleted_at
0,19099,2024-04-20 22:06:08+00:00,461009,follow,NaT
1,252634,2024-05-03 10:06:31+00:00,250833,follow,NaT
2,418937,2024-04-06 09:26:16+00:00,415186,follow,NaT
3,385938,2024-04-17 19:43:30+00:00,403156,follow,NaT
4,500406,2024-04-29 11:21:25+00:00,484567,follow,NaT


In [ ]:
G = nx.from_pandas_edgelist(links_df, source='fid', target='target_fid', create_using=nx.DiGraph)

In [ ]:


# Calculate PageRank
pagerank = nx.pagerank(G)

# Create a DataFrame with PageRank scores
pagerank_df = pd.DataFrame({'fid': list(pagerank.keys()), 'pagerank': list(pagerank.values())})

# Merge PageRank scores with verifications DataFrame
# verifications_with_pagerank = pd.merge(verifications_df, pagerank_df, on='fid', how='left')

# Now you can use the 'pagerank' column in verifications_with_pagerank to represent trust propagation.
# Higher PageRank scores indicate greater trust based on the network structure.

# print(verifications_with_pagerank.head())


In [ ]:
pagerank_df.head()

,fid,pagerank
0,19099,0.000012
1,461009,0.000003
2,252634,0.000007
3,250833,0.000001
4,418937,0.000002


In [ ]:
pagerank_df.sort_values(by='pagerank', ascending=False).head()

,fid,pagerank
1867,3,0.006092
834,5650,0.005517
177,99,0.004769
2696,2,0.004576
435,207,0.004384


In [ ]:
pagerank_df.to_parquet(f'/content/drive/My Drive/farcaster_bigquery/links/analytics/pagerank.parquet')
pagerank_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/analytics/pagerank.csv')
